In [1]:
import numpy as np
from collections import deque
from copy import deepcopy
import time

In [2]:
occupied_matrix = np.zeros((5,5), dtype = int)
occupied_matrix = occupied_matrix.astype(str)
answer_array = np.zeros((5,5), dtype = int)
answer_array = answer_array.astype(str)

In [3]:
def get_string(np_array):
    """
    Convert a np_array to a string
    To be used as immutable key
    """
    string = ""
    for i in range(5):
        for j in range(5):
            string += np_array[i][j]
    return string

In [4]:
### Find all variations of answers
answer_list = []
#purple
answer_array[1,4] = '8'
answer_array[2,2:5] = '8'
#red
answer_array[1, 2] = '3'
answer_array[0:2, 3] = '3'
#green
answer_array[0, 1:3] = '2'
answer_array[1, 1] = '2'
#orange
answer_array[1:3, 0] = '1'
answer_array[3:5, 0] = '4'
#yellow
answer_array[4, 1:3] = '7'
answer_array[4, 3:5] = '9'
#blue
answer_array[3, 3] = '5'
answer_array[3, 4] = '6'

answer_list.append(answer_array)
#Change 9 and 7
new_array = answer_list[0].copy()
new_array[4, 1:3] = '9'
new_array[4, 3:5] = '7'
answer_list.append(new_array)

#Change 1 and 4
new_list = [x.copy() for x in answer_list]
for np_array in new_list:
    np_array[1:3, 0] = '4'
    np_array[3:5, 0] = '1'
    answer_list.append(np_array)

#change 5 and 6
new_list =[x.copy() for x in answer_list]
for np_array in new_list:
    np_array[3,3] = '6'
    np_array[3,4] = '5'
    answer_list.append(np_array)

answer_list_strings = []
for answer in answer_list:
    answer_list_strings.append(get_string(answer))
answer_list_strings


['0223012338108884005647799',
 '0223012338108884005649977',
 '0223042338408881005617799',
 '0223042338408881005619977',
 '0223012338108884006547799',
 '0223012338108884006549977',
 '0223042338408881006517799',
 '0223042338408881006519977']

In [5]:
#shape class stores size, number, coordinates list for every shape
#the class changes the cordinates according to input matrix, if feasible
class shape:
    def __init__(self, start_pos : list, number : int):
        self.curr_pos = start_pos
        self.number = str(number)
        self.size = len(start_pos)

    def movements_U(self, matrix):
        new_position = []
        for i in range(self.size):
            new_position.append([self.curr_pos[i][0] - 1, self.curr_pos[i][1]])
            matrix[self.curr_pos[i][0] - 1, self.curr_pos[i][1]] = self.number
        for i in range(self.size):
            if self.curr_pos[i] not in new_position:
                matrix[self.curr_pos[i][0], self.curr_pos[i][1]] = '0'
            self.curr_pos[i][0] -= 1

    def movements_D(self, matrix):
        new_position = []
        for i in range(self.size):
            new_position.append([self.curr_pos[i][0] + 1, self.curr_pos[i][1]])
            matrix[self.curr_pos[i][0] + 1, self.curr_pos[i][1]] = self.number
        for i in range(self.size):
            if self.curr_pos[i] not in new_position:
                matrix[self.curr_pos[i][0], self.curr_pos[i][1]] = '0'
            self.curr_pos[i][0] += 1

    def movements_R(self, matrix):
        new_position = []
        for i in range(self.size):
            new_position.append([self.curr_pos[i][0], self.curr_pos[i][1] + 1])
            matrix[self.curr_pos[i][0], self.curr_pos[i][1] + 1] = self.number
        for i in range(self.size):
            if self.curr_pos[i] not in new_position:
                matrix[self.curr_pos[i][0], self.curr_pos[i][1]] = '0'
            self.curr_pos[i][1] += 1

    def movements_L(self, matrix):
        new_position = []
        for i in range(self.size):
            new_position.append([self.curr_pos[i][0], self.curr_pos[i][1] - 1])
            matrix[self.curr_pos[i][0], self.curr_pos[i][1] - 1] = self.number
        for i in range(self.size):
            if self.curr_pos[i] not in new_position:
                matrix[self.curr_pos[i][0], self.curr_pos[i][1]] = '0'
            self.curr_pos[i][1] -= 1

    def copy(self):
        return deepcopy(self)

In [6]:
def is_valid_move(shape, movement, matrix):
    valid_move = True
    if movement == 'U':
        for tup in shape.curr_pos:
            if tup[0] - 1 < 0:
                valid_move = False
                break
            if matrix[tup[0]-1][tup[1]] != '0' and matrix[tup[0]-1][tup[1]] != shape.number:
                valid_move = False
                break

    if movement == 'D':
        for tup in shape.curr_pos:
            if tup[0] + 1 > 4:
                valid_move = False
                break
            if matrix[tup[0]+1][tup[1]] != '0' and matrix[tup[0]+1][tup[1]] != shape.number:
                valid_move = False
                break

    if movement == 'R':
        for tup in shape.curr_pos:
            if tup[1] + 1 > 4:
                valid_move = False
                break
            if matrix[tup[0]][tup[1]+1] != '0' and matrix[tup[0]][tup[1]+1] != shape.number:
                valid_move = False
                break

    if movement == 'L':
        for tup in shape.curr_pos:
            if tup[1] - 1 < 0:
                valid_move = False
                break
            if matrix[tup[0]][tup[1]-1] != '0' and matrix[tup[0]][tup[1]-1] != shape.number:
                valid_move = False
                break
    return valid_move

In [7]:
#State score, configuration (in the form of string), shape_list, path leading to that state
class state:
    def __init__(self, matrix, shape_list, score, path):
        self.matrix_str = get_string(matrix)
        self.shape_list = shape_list
        self.score = score
        self.path = path

    def __repr__(self):
        return self.matrix_str

    def get_matrix(self):
        return np.array(list(self.matrix_str)).reshape((5,5))

In [8]:
def movements(shape, movement, score, path, matrix):
    """
    Movements takes shape, movement(L,R,U,D), score, path, matrix (for shape's methods)
    """
    if movement == 'U':
        shape.movements_U(matrix)
    if movement == 'D':
        shape.movements_D(matrix)
    if movement == 'L':
        shape.movements_L(matrix)
    if movement == 'R':
        shape.movements_R(matrix)
    path += (shape.number + movement)
    score += (5-shape.size)
    return score, path

In [9]:
#Initialize all the shapes
shape_1 = shape([[0,0], [1,0]], 1)
shape_2 = shape([[0,1], [1,1], [0,2]], 2)
shape_3 = shape([[0,3], [1,2], [1,3]], 3)
shape_4 = shape([[0,4], [1,4]], 4)
shape_5 = shape([[2,0]], 5)
shape_6 = shape([[2,1]], 6)
shape_7 = shape([[2,2], [2,3]], 7)
shape_8 = shape([[2,4], [3,2], [3,3], [3,4]], 8)
shape_9 = shape([[3,0], [3,1]], 9)

In [10]:
shape_list = []
shape_list.append(shape_8) #size 4
shape_list.append(shape_3) #size 3
shape_list.append(shape_2) #size 3
shape_list.append(shape_9) #size 2
shape_list.append(shape_7) #size 2
shape_list.append(shape_4) #size 2
shape_list.append(shape_1) #size 2
shape_list.append(shape_6) #size 1
shape_list.append(shape_5) #size 1

In [11]:
#Fill the empty matrix (Initial State)
for shape in shape_list:
    shape_cordi = shape.curr_pos
    for tup in shape_cordi:
        x = tup[0]
        y = tup[1]
        occupied_matrix[x][y] = shape.number

In [12]:
#Check if every is going as expected
occupied_matrix

array([['1', '2', '2', '3', '4'],
       ['1', '2', '3', '3', '4'],
       ['5', '6', '7', '7', '8'],
       ['9', '9', '8', '8', '8'],
       ['0', '0', '0', '0', '0']], dtype='<U21')

In [13]:
#state_map = {'np_string' : state_class}
#create the first state
first_state = state(occupied_matrix.copy(), shape_list, 0, '')
state_map = {}
state_map[repr(first_state)] = first_state
state_map_id_list = deque([repr(first_state)])

print(state_map[repr(first_state)].get_matrix())

[['1' '2' '2' '3' '4']
 ['1' '2' '3' '3' '4']
 ['5' '6' '7' '7' '8']
 ['9' '9' '8' '8' '8']
 ['0' '0' '0' '0' '0']]


In [14]:
t1 = time.time()
#The idea is to get all permutations of every state, while updating score and path for the new_state
#We used a map for storing all the states, and a deque to store keys of that map
#To speed up the algorithm, we only added new_state if they were not previously deleted (those states whoes permutations are already computed)
#If we reach same state, we compare the lowest score, and the state with lower score is stores in map
activity = ['L', 'R', 'U', 'D']
redundant_states = {}
found = False
count = 0
while len(state_map_id_list) != 0:
    count += 1
    if count % 10000 == 0:
        print(len(state_map_id_list))
    parent_id = state_map_id_list.popleft()
    parent_score = state_map[parent_id].score
    parent_path = state_map[parent_id].path
    parent_matrix = state_map[parent_id].get_matrix()
    parent_shape_list = state_map[parent_id].shape_list
    for shape_idx in range(9):
        for act in activity:
            if not is_valid_move(parent_shape_list[shape_idx], act, parent_matrix):
                continue

            new_shape_list = [x.copy() for x in parent_shape_list]
            #Create occupied matrix
            new_matrix = parent_matrix.copy()
            score = parent_score
            path = parent_path
            new_score, new_path = movements(new_shape_list[shape_idx], act, score, path, new_matrix)

            #Skip states with score more than 100
            if new_score > 150:
                continue

            new_state = state(new_matrix, new_shape_list, new_score, new_path)
            new_state_id = repr(new_state)

            #already processed and deleted
            if new_state_id in redundant_states:
                continue

            if new_state_id in state_map:
                #identical state with lower score
                if new_score < state_map[new_state_id].score:
                    state_map[new_state_id] = new_state
            else:
                state_map[new_state_id] = new_state
                state_map_id_list.append(new_state_id)

            #Finally we see if our current state matches any of our answers
            if new_state_id in answer_list_strings:
                found = True
                break
        if found: break
    if found: break
    redundant_states[parent_id] = 1
    del state_map[parent_id]
t2 = time.time()
print(f"Solving took {t2 - t1} seconds")

4029
6558
8650
10376
12157
14026
15679
17613
18940
20107
21036
22142
22770
23780
24170
25094
25725
26245
27235
27858
28837
29336
30171
31275
31838
32829
33861
34443
35358
35846
37129
37886
38814
39549
40392
41499
42290
42819
43527
44556
45594
46039
46509
47143
48414
48961
49390
49868
50740
51889
52518
53114
53815
54666
55930
56700
57307
58307
58690
60219
61041
62052
62599
63152
64289
65832
66655
67738
68251
69027
69773
71458
72782
73455
74477
74680
75797
76714
78388
80407
80834
81984
82435
83054
84192
84753
86776
88617
89063
90193
90899
91199
92166
93059
94406
96230
97531
97924
99409
100370
100792
101810
102849
103882
105175
106777
107537
108558
109856
110923
111501
112461
113593
114252
115809
117037
118346
118802
120017
121256
122390
122825
124052
124962
126206
127128
128221
129529
130443
130882
131986
133339
134050
134285
135143
136382
137506
138119
139053
139934
141117
142126
142478
143530
145043
145713
146094
146566
147480
148281
149477
149837
150779
151494
152762
153704
154092
154

In [15]:
print(found)

True


In [16]:
repr(new_state)

'0223042338408881005617799'

In [17]:
new_state.get_matrix()

array([['0', '2', '2', '3', '0'],
       ['4', '2', '3', '3', '8'],
       ['4', '0', '8', '8', '8'],
       ['1', '0', '0', '5', '6'],
       ['1', '7', '7', '9', '9']], dtype='<U1')

In [18]:
new_state.score

150

In [19]:
new_state.path

'8D9D7D7L4D6R5R1D1D2L3L4L4D3R3R2R2R1U7L1U5L6L4L3D2R4U4U4L2L3L8U9R9R9R7D7R6D5R1D1D4L2L3U8U1D4D6R6R6R5D5R5R'